In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(r'../input/breastcancerwisconsin/bc2.csv')

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.shape

In [ ]:
#lets check the dtypes
df.info()
#only one 'Bare Nuclei' object type feature remainig all are int type  

In [ ]:
#Lets analyse the Bare Nuclei 

print(df['Bare Nuclei'].value_counts())
# we can see that the values are numberical but there special character '?' assigned 
# we will handle this value by replacing this with median value
df['Bare Nuclei'] = df['Bare Nuclei'].replace('?',np.nan)
df['Bare Nuclei'].fillna(df['Bare Nuclei'].median(),inplace=True)

df['Bare Nuclei'] = df['Bare Nuclei'].astype('int')
print(df['Bare Nuclei'].value_counts())


In [ ]:
#check for the null values 
df.isnull().sum()
df.isnull().apply(pd.value_counts)

In [ ]:
df.describe().transpose()

EDA

In [ ]:
df.head()

In [ ]:
# we can remove the ID column as it has less relvance
df.drop('ID',axis=1,inplace=True)

In [ ]:
df.head()

In [ ]:
sns.countplot(df['Class'])

In [ ]:
df['Class'].value_counts()

In [ ]:
plt.figure(figsize=(20,6))
pos=1
for i in df.columns:
    plt.subplot(2,5,pos)
    sns.boxplot(df[i],orient= "v")
    plt.title(i)
    pos +=1
   
    
    

In [ ]:
#lets see the the outliers %

for k, v in df.items():
        q1 = v.quantile(0.25)
        q3 = v.quantile(0.75)
        irq = q3 - q1
        v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
        perc = np.shape(v_col)[0] * 100.0 / np.shape(df)[0]
        print("Column %s outliers = %.2f%%" % (k, perc))

In [ ]:
#Reducing the outliers 
for i in df.columns:
    q1,q2,q3 = df[i].quantile([0.25,0.50,0.75])
    iqr  = q3-q1
    a = df[i]>=q3+1.5*iqr
    b = df[i]<=q1-1.5*iqr
    df[i] = np.where(a|b,q2,df[i])


In [ ]:
#After removong the outlier lets see the differnce 
for k, v in df.items():
        q1 = v.quantile(0.25)
        q3 = v.quantile(0.75)
        irq = q3 - q1
        v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
        perc = np.shape(v_col)[0] * 100.0 / np.shape(df)[0]
        print("Column %s outliers = %.2f%%" % (k, perc))
        
#We can see the outlier are incresing 

In [ ]:
#lets see the skweness 
pos=1
plt.figure(figsize=(20,6))
for i in df.columns:
    plt.subplot(2,5,pos)
    sns.distplot(df[i],pos)
    pos +=1
    plt.title(i)
  

In [ ]:
#lets try to reduce the skewness


In [ ]:
x = df.iloc[:,:-1]
y = df['Class']

In [ ]:

for col in x.columns:
    if np.abs(x[col].skew()) > 0.3:
        x[col] = np.log1p(x[col])

In [ ]:
#correlation 
corr = df.corr()

In [ ]:
plt.figure(figsize=(20,6))
sns.heatmap(corr[(corr>=0.5)|(corr<=-0.4)],vmax=1,vmin=-1,annot=True,cmap='viridis')

In [ ]:
df.drop('Class',axis=1).corrwith(df['Class']).plot(kind='bar',title='Corr with Class')

In [ ]:
#Cell shape, cell size and Bare Nuclei are major factors in class detection
#cell shape and cell size are highly corr so we can consider one feature 
#Mitoses doesnot have any corr with class

In [ ]:
#Bivaraint analysis

In [ ]:

sns.jointplot(x=df['Cell Shape'],y = df['Cell Size'],kind='scatter')

In [ ]:
sns.violinplot(x=df['Cell Shape'],y=df['Cell Size'],hue=df['Class'])

In [ ]:
sns.barplot(x=df['Cell Shape'],y=df['Cell Size'],hue=df['Class'])

In [ ]:
from sklearn.neighbors import LocalOutlierFactor

In [ ]:
#-1 is that outlier is present

In [ ]:
lof = LocalOutlierFactor()
lof.fit_predict(x)[0:10]
x_score = lof.negative_outlier_factor_
outlier_score= pd.DataFrame()
outlier_score["score"]=x_score

lofthreshold= -2.5
loffilter= outlier_score["score"]< lofthreshold
outlier_index = outlier_score[loffilter].index.to_list()
x = x.drop(outlier_index)
y = y.drop(outlier_index).values

In [ ]:
x.shape,y.shape

In [ ]:
sns.regplot(x=df['Cell Shape'],y=df['Cell Size'],)

In [ ]:
sns.regplot(x=df['Cell Shape'],y=df['Bare Nuclei'])

In [ ]:
sns.regplot(x=df['Cell Size'],y=df['Bare Nuclei'])

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scl = StandardScaler()

In [ ]:
x = pd.DataFrame(scl.fit_transform(x),columns=x.columns)

In [ ]:
x['Mitoses'].value_counts()

In [ ]:
# we can remove the Mitoses feature 
x.drop('Mitoses',axis=1,inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.3)

In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
clf = LogisticRegression()

In [ ]:
clf.fit(xtrain,ytrain)

In [ ]:
pred = clf.predict(xtest)

In [ ]:
from sklearn.metrics import confusion_matrix,accuracy_score

In [ ]:
train_score = clf.score(xtrain,ytrain)
test_score = accuracy_score(ytest,pred)
tn,fp,fn,tp=confusion_matrix(ytest,pred).ravel()
recall = round(tp/(tp+fn),3)
specificity = round(tn/(tn+fp),3)
precision = round(tp/(tp+fp),3)


In [ ]:
final = pd.DataFrame({'Model':['Logistic Reg'],'accuracy':[test_score],'True +ve rate':[recall],'True -ve Rate':[specificity],'precision':[precision]})

In [ ]:
final

In [ ]:
#lets try to improve the accuracy

In [ ]:
y = pd.DataFrame(y)

In [ ]:
y.value_counts()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier()
tree.fit(xtrain, ytrain)

pred = tree.predict(xtest)

train_score = tree.score(xtrain,ytrain)
test_score = accuracy_score(ytest,pred)
tn,fp,fn,tp=confusion_matrix(ytest,pred).ravel()
recall = round(tp/(tp+fn),3)
specificity = round(tn/(tn+fp),3)
precision = round(tp/(tp+fp),3)
final.loc[1] = ['Decision Tree',test_score,recall,specificity,precision]
final

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf = RandomForestClassifier()
rf.fit(xtrain,ytrain)
pred = rf.predict(xtest)
train_score = rf.score(xtrain,ytrain)
test_score = accuracy_score(ytest,pred)
tn,fp,fn,tp=confusion_matrix(ytest,pred).ravel()
recall = round(tp/(tp+fn),3)
specificity = round(tn/(tn+fp),3)
precision = round(tp/(tp+fp),3)
final.loc[2] = ['Random Forest',test_score,recall,specificity,precision]
final

In [ ]:
#lets try using Cross validastion

from sklearn.model_selection import cross_val_score

In [ ]:
cv = cross_val_score(clf,x,y,cv=5,n_jobs=4,verbose=2).mean()
cv1 = cross_val_score(tree,x,y,cv=5,n_jobs=4,verbose=2).mean()
cv2 = cross_val_score(rf,x,y,cv=5,n_jobs=4,verbose=2).mean()
print(cv,cv1,cv2)

Cross validation with Random forest had given better accuracy of 96%

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC()#rbf by default

In [ ]:
svm.fit(xtrain,ytrain)

In [ ]:
pred = svm.predict(xtest)
train_score = svm.score(xtrain,ytrain)
test_score = accuracy_score(ytest,pred)
tn,fp,fn,tp=confusion_matrix(ytest,pred).ravel()
recall = round(tp/(tp+fn),3)
specificity = round(tn/(tn+fp),3)
precision = round(tp/(tp+fp),3)
final.loc[3] = ['SVM',test_score,recall,specificity,precision]
final

In [ ]:
#Lets see if we further tune the RF and accuracy has imporved or not

In [ ]:
model_params  = {
    "svm" : {
        "model":SVC(gamma="auto"),
        "params":{
            'C' : [1,10,20],
            'kernel':['poly','linear','sigmoid']
        }
    },
    
    "decision_tree":{
        "model": DecisionTreeClassifier(),
        "params":{
            'criterion':["entropy","gini"],
            "max_depth":[5,8,9]
        }
    },
    
    "random_forest":{
        "model": RandomForestClassifier(),
        "params":{
            "n_estimators":[1,5,10],
            "max_depth":[5,8,9]
        }
    },
   
    
    'logistic_regression' : {
        'model' : LogisticRegression(solver='liblinear',multi_class = 'auto'),
        'params': {
            "C" : [1,5,10]
        }
    }
    
}


In [ ]:
score=[]
from sklearn.model_selection import GridSearchCV
for model_name,mp in model_params.items():
    clf = GridSearchCV(mp["model"],mp["params"],cv=8,return_train_score=False)
    clf.fit(x,y)
    score.append({
        "Model" : model_name,
        "Best_Score": clf.best_score_,
        "Best_Params": clf.best_params_
    })
df5 = pd.DataFrame(score,columns=["Model","Best_Score","Best_Params"])


In [ ]:
df5

In [ ]:
svm_tuned = SVC(C=1,kernel='linear')#rbf by default
svm_tuned.fit(xtrain,ytrain)

In [ ]:
pred = svm_tuned.predict(xtest)
train_score = svm_tuned.score(xtrain,ytrain)
test_score = accuracy_score(ytest,pred)
tn,fp,fn,tp=confusion_matrix(ytest,pred).ravel()
recall = round(tp/(tp+fn),3)
specificity = round(tn/(tn+fp),3)
precision = round(tp/(tp+fp),3)
final.loc[4] = ['SVM_tuned',0.963,recall,specificity,precision]
final

Tune SVM has imnpored the accuracy to 96.3% whne Linear Kernel was used